# Wypożyczalnia sprzętu sportowego: Model relacyjnej bazy danych

**Autor**: Filip Dąbrowski </br>
**Numer indeksu**: 341057 </br>
**Wydział**: Elektryczny </br>
**Przedmiot**: Bazy Danych </br>

## Opis projektu

Poniżej przedstawiono proces implementacji bazy danych dla wypożyczalni sprzętu sportowego, zgodny z wymaganiami projektu udostępnionymi na platformie Leia.

Do modelowania wykorzystałem narzędzie **Vertabelo**, zgodnie z rekomendacją przedstawioną podczas zajęć. Implementację bazy danych przeprowadziłem lokalnie w środowisku **MS SQL Server**.

Podczas realizacji projektu zastosowałem kompleksową wiedzę zdobytą w trakcie kursu, uwzględniając następujące elementy:
1. **DCL** – użyłem do utworzenia użytkownika i nadania mu odpowiednich uprawnień.
2. **DDL** – wykorzystałem do stworzenia struktury bazy danych, w tym tabel, kluczy i indeksów.
3. **DQL** – zastosowałem w trakcie definiowania procedur składowanych.
4. **DTL** – wprowadziłem transakcje w jednej z procedur składowanych, aby zapewnić spójność danych.
5. **DML** – używany w procedurach składowanych do manipulacji danymi w bazie.

Dodatkowo, udostępniam publiczne repozytorium z kodem na github: https://github.com/filipdbr/bazy_danych_projekt_zaliczeniowy

## Modelowanie

### Model konceptualny

Model konceptualny jest modelem uproszczony. Nie wskazałem w nim kluczy obcych ani relacji wiele-do-wielu, a także zastosowałem tylko minimalną normalizację. Modelowanie konceptualne na początku projektu pomogło mi określić, jakie encje będą potrzebne, jakie mniej więcej będą miały atrybuty i jak będą się ze sobą łączyć.

![Model Konceptualny](https://github.com/filipdbr/bazy_danych_projekt_zaliczeniowy/blob/main/wypozyczalnia_model_konceptualny.png)

### Model implementacyjny

![Model Implementacyjny](https://github.com/filipdbr/bazy_danych_projekt_zaliczeniowy/blob/main/wypozyczalnia_model_implementacyjny.png)

Model podzieliłem na pięć obszarów biznesowych, według których później stworzyłem schematy. Są to:
* Osoby,
* Zamówienia,
* Serwis,
* Sprzęt,
* Kategorie.

Taki podział umożliwa w systemie odpowiednie nadanie uprawnień, zwiększając bezpieczeństwo i integralność danych. Naturalnie, podział na schematy mógłby być inny i zależy od specyfiki biznesu.

#### Osoby

Klientów i pracowników postanowiłem podzielić na dwie oddzielne tabele, które powiązałem z tabelą nadrzędną `Osoba`. Każdy pracownik i klient musi być osobą. Osoby mogą istnieć niezależnie i nie muszą być przypisane do żadnej z encji podrzędnych. W takim układzie możliwe jest także, że jedna osoba jest jednocześnie pracownikiem i klientem. Takie rozwiązanie było moją świadomą decyzją i zamierzonym działaniem systemu.

Alternatywnie mogłem stworzyć jedną dużą tabelę zawierającą dane zarówno klientów, jak i pracowników. Uznałem jednak, że taka tabela zawierałaby zbyt wiele wartości `NULL`, ponieważ encje podrzędne mają mało wspólnych atrybutów.

W encji `Pracownik` zastosowałem relację unarną, aby uwzględnić przełożonego w tej samej tabeli. Według mojej wiedzy, takie rozwiązanie jest powszechnie stosowane. **Świadomie zdecydowałem się nie normalizować atrybutu `Adres`**. Przyjąłem, że liczba pracowników wypożyczalni będzie stosunkowo niewielka, a czas na przygotowanie projektu miałem ograniczony, dlatego zostawiłem to pole w formie tekstu. Znormalizowałem jednak atrybuty `StatusZatrudnienia` oraz `Stanowisko`, aby uniknąć redundancji danych.

#### Zamówienia

Zamówienia stanowią niewielką (lecz kluczową) część modelu. Zgodnie z założeniami, zamówienie może być związane z wieloma (maksymalnie pięcioma) egzemplarzami. Egzemplarz może być przypisany do wielu zamówień, co tworzy relację wiele-do-wielu. Rozwiązałem to za pomocą tabeli pośredniczącej `ZamowienieEgzemplarz`. Ograniczenie do pięciu egzemplarzy jest niemożliwe w modelu ERD - wprowadziłem je później za pomocą wyzwalacza.

#### Serwis

Reklamacje w moim modelu muszą być przypisane do jednego, konkretnego zamówienia, ale mogą obejmować wiele egzemplarzy. Nie wprowadziłem logiki wymuszającej, aby egzemplarz w reklamacji musiał być częścią przypisanego zamówienia. Można to osiągnąć poprzez odpowiedni wyzwalacz lub procedurę składowaną. Gdybym miał więcej czasu, zaimplementowałbym takie rozwiązanie. W celu uniknięcia redundancji wprowadziłem osobną encję `StatusReklamacji`.

#### Sprzęt

Jeśli chodzi o sprzęt, zdecydowałem się na jedną tabelę dla całego sprzętu z atrybutem określającym, czy sprzęt jest profesjonalny. Uważałem, że **sprzętu profesjonalnego będzie znacząco mniej niż amatorskiego**, dlatego takie rozwiązanie wydaje się optymalne. Dodałem również pola `Rabat` i `Opis`, nawet dla sprzętu profesjonalnego, mimo że nie było to wymagane w projekcie. Wychodzę z założenia, że jeżeli sprzęt nie ma przypisanego 1 w atrybucie `profesjonalny` w encji `Sprzet`, oznacza to, że jest amatorski.

W projekcie typ sprzętu nazwano "typem", ja jednak uznałem, że lepszym określeniem będzie "kategoria". Na przykład "górski" to zarówno typ, jak i kategoria sprzętu. W ten sposób można przypisać uprawnienia zarówno do sprzętu profesjonalnego, jak i do kategorii. Przyjąłem, że uprawnienia mogą nie wystąpić i może istnieć sprzęt lub kategoria bez przypisanych uprawnień.

#### Kategoria

Kategorie w moim modelu mogą być przypisane do wielu pór roku, a pora roku do wielu kategorii. Relację wiele-do-wielu wyeliminowałem za pomocą tabeli pośredniczącej `KategoriaPoraRoku`. 

`Ranking` przypisałem w tabeli pośredniczącej pomiędzy `Sprzęt` a `Kategoria`, klasyfikując go jako należący do schematu `Sprzęt`. Uznałem to za bardziej sensowne z biznesowego punktu widzenia, ponieważ ranking przypisuje się do każdego sprzętu i kategorii.

#### Największe wyzwania przy modelowaniu
Największym wyzwaniem było dla podjęcie decyzji o tym, w jaki sposób wymodelować encje `Klient` i `Pracownik`. Czy użyć jednej tabeli czy kliku (zadawałem zrzesztą to pytanie na forum przedniotu). Poza tym, wymodelowanie typów sprzętu takich jak profesjonalny, amatorski, górski i wodny, również zajęło mi sporo czasu. 

## Implementacja modelu
Implementację bazy danych przeprowadziłem lokalnie w środowisku **MS SQL Server**.

### Tworzenie bazy danych

#### Usunięcie takiej bazy danych o takiej samej nazwie (jeżeli istnieje)

In [ ]:
IF EXISTS (SELECT * FROM master..sysdatabases WHERE NAME = 'Wypozyczalnia')
DROP DATABASE Wypozyczalnia;
GO

#### Utworzenie bazy danych wraz z jawnym wskazaniem lokalizacji

In [ ]:
CREATE DATABASE Wypozyczalnia
ON Primary
(	name = WypozyczalniaData,
	filename = 'C:\baza_danych_wypozyczalnia\data\Wypozyczalnia.mdf',
	size = 20MB,
	maxsize = 100MB,
	filegrowth = 10MB)
LOG ON
(	name = WypozyczalniaLog,
	filename = 'C:\baza_danych_wypozyczalnia\log\Wypozyczalnia.mdf',
	size = 10MB,
	maxsize = 50MB,
	filegrowth = 10MB)

#### Utworznie użytkownika

In [ ]:
USE Wypozyczalnia;
GO

-- usunięcie użytkownika jeżeli istnieje 
IF EXISTS (SELECT * FROM sys.database_principals WHERE name = 'admin_wypozyczalnia')
    DROP USER admin_wypozyczalnia;
GO

-- utworzenie użytkownika
CREATE USER admin_wypozyczalnia FOR LOGIN admin_wypozyczalnia;

#### Utworzenie schematów

In [ ]:
GO
CREATE SCHEMA Osoby;
GO
CREATE SCHEMA Sprzet;
GO
CREATE SCHEMA Zamowienia;
GO
CREATE SCHEMA Kategorie;
GO
CREATE SCHEMA Serwis;
GO

#### Nadanie uprawnień
Nadanie uprawnień oraz przypisanie roli `db_owner` dla użytkownika `admin_wypozyczalnia`

In [ ]:
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Osoby TO admin_wypozyczalnia;
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Sprzet TO admin_wypozyczalnia;
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Zamowienia TO admin_wypozyczalnia;
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Kategorie TO admin_wypozyczalnia;
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::Serwis TO admin_wypozyczalnia;

-- Przydzielenie użytkownika admin_wypozyczalnia do roli db_owner
ALTER ROLE db_owner ADD MEMBER admin_wypozyczalnia;
GO

### Tworzenie encji oraz indeksów
Ponieżej wklejam duży blok kodu tworzący wszystkie tabele w bazie danych oraz indeksy. Kod opisany jest komentarzami.

In [ ]:
USE Wypozyczalnia;
GO

-- encja 'Osoba'
CREATE TABLE Osoby.Osoba (
	
	-- Atrybuty
    osoba_id INT IDENTITY(1,1) NOT NULL,
    imie NVARCHAR(25) NOT NULL,
    nazwisko NVARCHAR(50) NOT NULL,
    telefon VARCHAR(15) NOT NULL,
    
	-- Klucz główny
    CONSTRAINT Osoba_pk PRIMARY KEY (osoba_id),
	
	-- Ograniczenia
    CHECK (LEN(imie) >= 2),
    CHECK (LEN(nazwisko) >= 2),
    CHECK (LEN(telefon) >= 7 AND LEN(telefon) <= 15),
	CHECK (telefon NOT LIKE '%[^0-9]%') -- telefon może składać się jedynie z cyfr
);

-- encja 'Klient'
CREATE TABLE Osoby.Klient (
	
	-- Atrybuty
	klient_id INT IDENTITY(1,1) NOT NULL,
	firma nvarchar(50),
	nip varchar(10),
	osoba_id INT NOT NULL,
	data_utworzenia date DEFAULT GETDATE(),

	-- Klucz główny i klucz obcy
	CONSTRAINT Klient_pk PRIMARY KEY (klient_id),	
	CONSTRAINT Osoba_fk FOREIGN KEY (osoba_id) REFERENCES Osoby.Osoba(osoba_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE, -- jeżeli usuniemy osobę, chcę usunąć wszystkie rekordy z nią związne
	
	-- Ograniczenia
	CHECK (firma IS NULL OR LEN(firma) >= 3)
);

-- encja 'Stanowisko'
CREATE TABLE Osoby.Stanowisko(

	-- Atrybuty
	stanowisko_id INT IDENTITY(1,1) NOT NULL,
	nazwa nvarchar(50) NOT NULL,

	-- Klucz główny
	CONSTRAINT Stanowisko_pk PRIMARY KEY (stanowisko_id),

	-- Ograniczenia
	CHECK(LEN(nazwa) > 3),
	CONSTRAINT Stanowisko_nazwa_uk UNIQUE (nazwa)
);

-- encja 'Status Zatrudnienia'
CREATE TABLE Osoby.StatusZatrudnienia (
    
	-- Atrybuty
    status_zatrudnienia_id INT IDENTITY(1,1) NOT NULL,
    nazwa_statusu NVARCHAR(30) NOT NULL,

    -- Klucz główny
    CONSTRAINT Status_zatrudnienia_pk PRIMARY KEY (status_zatrudnienia_id),

    -- Ograniczenia
    CHECK (LEN(nazwa_statusu) > 3 AND nazwa_statusu NOT LIKE '%[^A-Za-z]%')
);



-- encja 'Pracownik'
CREATE TABLE Osoby.Pracownik (
    
	-- Atrybuty
    pracownik_id INT IDENTITY(1,1) NOT NULL,
    przelozony_id INT DEFAULT 1,
    stanowisko_id INT DEFAULT 7 NOT NULL,
    pesel VARCHAR(11),
    adres NVARCHAR(100) NOT NULL,
    data_zatrudnienia DATE NOT NULL,
    status_zatrudnienia INT DEFAULT 1 NOT NULL,
    osoba_id INT,

    -- Klucze
    CONSTRAINT Pracownik_pk PRIMARY KEY (pracownik_id),
    CONSTRAINT Pracownik_Przełożony_fk FOREIGN KEY (przelozony_id) REFERENCES Osoby.Pracownik(pracownik_id)
		ON UPDATE NO ACTION
		ON DELETE NO ACTION, -- logika zostanie obsłużna przez wyzwalacz
    CONSTRAINT Pracownik_Stanowisko_fk FOREIGN KEY (stanowisko_id) REFERENCES Osoby.Stanowisko(stanowisko_id)
		ON UPDATE CASCADE
		ON DELETE SET DEFAULT, -- domyślnie stworzone jest stanowisko o ID 7, który nazywa się "do aktualizacji".
    CONSTRAINT Pracownik_Osoba_fk FOREIGN KEY (osoba_id) REFERENCES Osoby.Osoba(osoba_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
    CONSTRAINT Pracownik_Status_fk FOREIGN KEY (status_zatrudnienia) REFERENCES Osoby.StatusZatrudnienia(status_zatrudnienia_id)
		ON UPDATE CASCADE
		ON DELETE SET DEFAULT,

    -- Ograniczenia
	-- może być tylko 1 kierownik główny - zostanie to obsłużone przez trigger.
	-- tylko kierownik gł. może nie mieć przełożonego - zostanie to obsłużone przez trigger.
    CHECK (pesel IS NULL OR (LEN(pesel) = 11 AND pesel NOT LIKE '%[^0-9]%')),
    CHECK (LEN(adres) >= 5),
    CHECK (data_zatrudnienia >= '2001-01-01'),
);

CREATE INDEX index_przelozony ON Osoby.Pracownik(przelozony_id); -- do obsługi wyszukiwań pracowników przełożonego
CREATE UNIQUE INDEX UQ_Pracownik_Pesel ON Osoby.Pracownik(pesel) WHERE pesel IS NOT NULL; -- w ten sposób uzyskam unikalną wartośc pesel lub wiele wartości null


-- encja 'Zamówienie'
CREATE TABLE Zamowienia.Zamowienie(
	
	-- Atrybuty
	zamowienie_id INT IDENTITY(1,1) NOT NULL,
	cena_calkowita decimal(8,2) NOT NULL,
	klient_id INT NOT NULL,
	pracownik_id INT DEFAULT 1 NOT NULL,
	data_zamowienia DATE NOT NULL,

	-- Klucze
	CONSTRAINT Zamowienie_pk PRIMARY KEY (zamowienie_id),
	CONSTRAINT Klient_Zmowienie_fk FOREIGN KEY (klient_id) REFERENCES Osoby.Klient(klient_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	CONSTRAINT Pracownik_Zamowienie_fk FOREIGN KEY (pracownik_id) REFERENCES Osoby.Pracownik(pracownik_id)
		ON UPDATE NO ACTION		-- obsłużone zostanie przez procedurę składowaną
		ON DELETE NO ACTION,	-- obsłużone zostanie przez procedurę składowaną

	-- Ograniczenia
	CHECK (cena_calkowita > 0.00)
);

-- tworzę indexy dla encji 'Zamowienie'
CREATE INDEX index_klient_id On Zamowienia.Zamowienie(klient_id); -- bedziemy szukać liczby zamówień klientów
CREATE INDEX index_pracownik_id ON Zamowienia.Zamowienie(pracownik_id); -- będziemy szukać pracowników z największą liczbą zamówień
CREATE INDEX index_zamowienie_i_data_zamowienia ON Zamowienia.Zamowienie(zamowienie_id, data_zamowienia); -- dla wyszukiwania zamówień w zakresie dat


/* następnie muszę stworzyć encję 'Producent' i tworzyć kolejne encje w kierunku 'Zamówienie;.
Wynika to z tego, że aby tworzyć FK w tabeli podrzędnej, w pierwszej kolejności muszę stworzyć tabele nadrzędne.*/

-- encja 'Producent'
CREATE TABLE Sprzet.Producent (

    -- Atrybuty
    producent_id INT IDENTITY(1,1) NOT NULL,
    nazwa NVARCHAR(50) NOT NULL,
    email NVARCHAR(50) NOT NULL,
    telefon NVARCHAR(15) NOT NULL,
    jezyk NVARCHAR(15),
    data_utworzenia DATE DEFAULT GETDATE(),

    -- Klucze
    CONSTRAINT Producent_pk PRIMARY KEY (producent_id),

    -- Ograniczenia
    CHECK (LEN(nazwa) > 2 AND nazwa NOT LIKE '%[^A-Za-z0-9 ]%'),	-- dopuszczam w nazwie litery wielkie i małe, cyfry oraz spację
    CHECK (LEN(telefon) > 7 AND telefon NOT LIKE '%[^0-9]%'),		-- w numerze telefonu dopuszczam jedynie cyfry
    CHECK (LEN(jezyk) > 4 AND jezyk NOT LIKE '%[^A-Za-z]%')			-- w nazwie języka dopuszczam litery wielkie i małe
);

-- Indeks na atrybucie nazwa: możliwe częstwe wyszukiwanie po nazwie producenta
CREATE INDEX index_producent_nazwa ON Sprzet.Producent(nazwa);

-- encja 'Sprzet'
CREATE TABLE Sprzet.Sprzet(
	
	-- Atrybuty
	sprzet_id INT IDENTITY(1,1) NOT NULL,
	nazwa nvarchar(100) NOT NULL,
	cena_za_dobe decimal(8,2) NOT NULL,
	profesjonalny BIT,
	rabat decimal(5,2),
	opis nvarchar(255),
	producent_id INT NOT NULL,

	-- Klucze
	CONSTRAINT Sprzet_pk PRIMARY KEY (sprzet_id),
	CONSTRAINT Producent_Sprzet_fk FOREIGN KEY (producent_id) REFERENCES Sprzet.Producent(producent_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,

	-- Ograniczenia
	CHECK(LEN(nazwa) > 3),
	CHECK(cena_za_dobe > 0.00),
	CHECK(rabat >= 0.00)

);

-- encja 'Egzemplarz'
CREATE TABLE Sprzet.Egzemplarz (
    
	-- Atrybuty
    egzemplarz_id INT IDENTITY(1,1) NOT NULL,
    wypozyczony BIT NOT NULL DEFAULT 0, -- Domyślnie sprzęt nie jest wypożyczony
    sprzet_id INT NOT NULL,

    -- Klucze
    CONSTRAINT Egzemplarz_pk PRIMARY KEY (egzemplarz_id),
    CONSTRAINT Sprzet_Egzemplarz_fk FOREIGN KEY (sprzet_id) REFERENCES Sprzet.Sprzet(sprzet_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE
);

-- encja 'Reklamacja'
CREATE TABLE Serwis.Reklamacja(
	
	-- Atrybuty
	reklamacja_id INT IDENTITY(1,1) NOT NULL,
	zamowienie_id INT NOT NULL,
	powod nvarchar(255) NOT NULL,
	data_reklamacji date NOT NULL,

	-- Klucze
	CONSTRAINT Reklamacja_pk PRIMARY KEY (reklamacja_id),
	CONSTRAINT Reklamacja_Zamowienie_fk FOREIGN KEY (zamowienie_id) REFERENCES Zamowienia.Zamowienie(zamowienie_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE
);

-- encja zawierająca statusy reklamacji
CREATE TABLE Serwis.StatusReklamacji(

	-- Atrybuty
	status_reklamacji_id INT IDENTITY (1,1) NOT NULL,
	nazwa nvarchar(20) UNIQUE NOT NULL,

	-- Klucz główny
	CONSTRAINT StatusReklamacji_pk PRIMARY KEY (status_reklamacji_id)

);

-- encja asocjacyjna dla encji Reklamacja i Egzemplarz. Konieczna, ponieważ jedna reklamacja może dotyczyć wielu egzemplarzy
CREATE TABLE Serwis.ReklamacjaEgzemplarz(

	-- Atrybuty
	reklamacja_egzemplarz_id INT IDENTITY (1,1) NOT NULL,
	reklamacja_id INT NOT NULL,
	egzemplarz_id INT NOT NULL,
	status_reklamacji INT NOT NULL,

	-- Klucze
	CONSTRAINT ReklamacjaEgzemplarz_pk PRIMARY KEY (reklamacja_egzemplarz_id),
	CONSTRAINT Reklamacja_ReklamacjaEgzemplarz_fk FOREIGN KEY (reklamacja_id) REFERENCES Serwis.Reklamacja(reklamacja_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	CONSTRAINT Egzemplarz_ReklamacjaEgzemplarz_fk FOREIGN KEY (egzemplarz_id) REFERENCES Sprzet.Egzemplarz(egzemplarz_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	CONSTRAINT StatusReklamacji_ReklamacjaEgzemplarz_fk FOREIGN KEY (status_reklamacji) REFERENCES Serwis.StatusReklamacji(status_reklamacji_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,

);

-- encja 'Naprawa'
CREATE TABLE Serwis.Naprawa (
    
	-- Atrybuty
    naprawa_id INT IDENTITY(1,1) NOT NULL,
    data_zgloszenia DATE DEFAULT GETDATE(),
    termin_ukonczenia DATE DEFAULT NULL,
    powod VARCHAR(255),
    koszt_naprawy DECIMAL(8,2) CHECK (koszt_naprawy >= 0.00) DEFAULT NULL,
    egzemplarz_id INT NOT NULL,
	reklamacja_egzemplarz_id INT

    -- Klucze
    CONSTRAINT Naprawa_pk PRIMARY KEY (naprawa_id),
    CONSTRAINT Egzemplarz_Naprawa_fk FOREIGN KEY (egzemplarz_id) REFERENCES Sprzet.Egzemplarz(egzemplarz_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	CONSTRAINT ReklamacjaEgzemplarz_Naprawa_fk FOREIGN KEY (reklamacja_egzemplarz_id) REFERENCES Serwis.ReklamacjaEgzemplarz(reklamacja_egzemplarz_id)
		ON UPDATE NO ACTION
		ON DELETE NO ACTION,

    -- Ograniczenia
    CHECK (termin_ukonczenia IS NULL OR termin_ukonczenia >= data_zgloszenia)
);

-- encja asocjacyjna 'ZamowienieEgzemplarz'
CREATE TABLE Zamowienia.ZamowienieEgzemplarz (

	-- Atrybuty
	zamowienie_id INT NOT NULL,
	egzemplarz_id INT NOT NULL,
	data_zwrotu DATE NOT NULL,

	-- Klucze
	CONSTRAINT ZamowienieEgzemplarz_pk PRIMARY KEY (zamowienie_id, egzemplarz_id),
	CONSTRAINT Zamowienie_ZamowienieEgzemplarz_fk FOREIGN KEY (zamowienie_id) REFERENCES Zamowienia.Zamowienie(zamowienie_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE, -- w przypadku usunięcia rekodu w tabeli nadrzędnej usuwam wszystkie rekory również w tej tabeli
	CONSTRAINT Egzemplarz_ZamowienieEgzemplarz_fk FOREIGN KEY (egzemplarz_id) REFERENCES Sprzet.Egzemplarz(egzemplarz_id)
		ON UPDATE NO ACTION		-- obsłużone zostanie przez procedurę składowaną
		ON DELETE NO ACTION,	-- w przypadku usunięcia rekordu egzemplarz_id w tabeli nadrzędnej zarządzę logiką poprzez wyzwalacz
	CONSTRAINT ZamowienieEgzemplarz_UNIQUE UNIQUE (zamowienie_id, egzemplarz_id)	-- brak możliwości wstawienia do zamówienia tego samego egzemplarza 2 razy 

);

-- Encja 'Uprawnienie'
CREATE TABLE Sprzet.Uprawnienie (

	-- Atrybuty
    uprawnienie_id INT IDENTITY(1,1) NOT NULL,
    nazwa NVARCHAR(100) NOT NULL,

    -- Klucze
    CONSTRAINT Uprawnienie_pk PRIMARY KEY (uprawnienie_id)
);

-- Encja 'Sprzęt profesjonalny' przechowująca jedynie te modele, które klasyfikowane są jako profesjonalne
CREATE TABLE Sprzet.SprzetProfesjonalny (

	-- Atrybuty
    sprzet_id INT NOT NULL,
    uprawnienie_id INT,

    -- Klucze
    CONSTRAINT SprzetProfesjonalny_pk PRIMARY KEY (sprzet_id, uprawnienie_id),
    CONSTRAINT Sprzet_SprzetProfesjonalny_fk FOREIGN KEY (sprzet_id) REFERENCES Sprzet.Sprzet(sprzet_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
    CONSTRAINT Uprawnienie_SprzetProfesjonalny_fk FOREIGN KEY (uprawnienie_id) REFERENCES Sprzet.Uprawnienie(uprawnienie_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE
);

-- Encja 'Kategoria' przechowujące informace o kategoriach sprzętu w systemie
CREATE TABLE Kategorie.Kategoria (

	-- Atrybuty
    kategoria_id INT IDENTITY(1,1) NOT NULL,
    nazwa NVARCHAR(50) NOT NULL,
    uprawnienie_id INT,

    -- Klucze
    CONSTRAINT Kategoria_pk PRIMARY KEY (kategoria_id),
	CONSTRAINT Uprawnienie_Kategoria FOREIGN KEY (uprawnienie_id) REFERENCES Sprzet.Uprawnienie(uprawnienie_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE
);

-- Encja asocjacyjna dla encji 'Sprzet' i 'Kategoria'. Pozwala obsłużyć relację wiele-do-wielu. Dodatkowo wprowadza ranking.
CREATE TABLE Sprzet.SprzetKategoria (

	-- Atrybuty
    sprzet_id INT NOT NULL,
    kategoria_id INT DEFAULT 1 NOT NULL,
    ranking INT NOT NULL,

    -- Klucze
    CONSTRAINT SprzetKategoria_pk PRIMARY KEY (sprzet_id, kategoria_id),
    CONSTRAINT Sprzet_SprzetKategoria_fk FOREIGN KEY (sprzet_id) REFERENCES Sprzet.Sprzet(sprzet_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
    CONSTRAINT Kategoria_SprzetKategoria_fk FOREIGN KEY (kategoria_id) REFERENCES Kategorie.Kategoria(kategoria_id)
		ON UPDATE CASCADE
		ON DELETE SET DEFAULT,

	-- Ograniczenia
	CONSTRAINT Ranking_uq UNIQUE (kategoria_id, ranking), -- w rankingu nie może być miejsc ex aequo. Kombinacja miejsce ranking musi być unikatowa
	CHECK (ranking >= 1)
);

CREATE INDEX index_ranking_kategoria ON Sprzet.SprzetKategoria(kategoria_id, ranking);
CREATE INDEX index_sprzet_kategoria ON Sprzet.SprzetKategoria(sprzet_id);
CREATE INDEX index_kategoria_sprzetkategoria ON Sprzet.SprzetKategoria(kategoria_id);


-- Encja 'PoraRoku'. Dopuszam inne nazwy pór roku niż standrdowe 4, stąd brak ograniczeń. Może być to np. przedwiośnie.
CREATE TABLE Kategorie.PoraRoku (

	-- Atrybuty
    pora_roku_id INT IDENTITY(1,1) NOT NULL,
    nazwa NVARCHAR(20) NOT NULL,

    -- Klucze
    CONSTRAINT PoraRoku_pk PRIMARY KEY (pora_roku_id)
);

-- Kolejna tabela pośrednicząca, tym razem dla Pory Roku i Kategorii. Kategoria, może być przypisana do wielu pór roku. 
CREATE TABLE Kategorie.KategoriaPoraRoku (

	-- Atrybuty
    kategoria_id INT NOT NULL,
    pora_roku_id INT DEFAULT 1 NOT NULL,

    -- Klucze
    CONSTRAINT KategoriaPoraRoku_pk PRIMARY KEY (kategoria_id, pora_roku_id),
    CONSTRAINT Kategoria_KategoriaPoraRoku_fk FOREIGN KEY (kategoria_id) REFERENCES Kategorie.Kategoria(kategoria_id)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
    CONSTRAINT PoraRoku_KategoriaPoraRoku_fk FOREIGN KEY (pora_roku_id) REFERENCES Kategorie.PoraRoku(pora_roku_id)
		ON UPDATE CASCADE
		ON DELETE SET DEFAULT
);

## Implementacja wyzwalaczy
Tworzę wyzwalacze które pomagają wprowadzić dodatkową logikę, której nie udało mi się osiągnąć poprzez relacje oraz `CHECK()`. W realnym systemie, stowrzyłbym wiele wyzwalaczy weryfikujących daty (np. data zwrotu >= data wypożyczenia). Nie mogłem jednak tego zrobić przez bardzo duży problem z wygenerowaniem danych testowych, a konkretniej dat, które spełniałyby warunki. 

### Trigger: Walidacja danych w tabeli `Pracownik`

Wyzwalacz został stworzony w celu zapewnienia integralności danych w tabeli `Osoby.Pracownik`. Składa się z 3 etapów/funkcjonalności:

1. **Zapewnienie, że tylko jeden pracownik ma `stanowisko_id = 1`**  
   Trigger sprawdza, czy więcej niż jeden pracownik o `stanowisko_id = 1`. Jeśli taki przypadek wystąpi, operacja jest cofana. Wynika to z tego, że tylko 1 pracownik może być Głównym Kierownikiem (`stanowisko_id = 1`).

2. **Sprawdzenie przypisania przełożonego dla innych stanowisk**  
   Dla pracowników o `stanowisko_id` innym niż 1, wymuszana jest posiadanie przełożonego. Jeśli `przelozony_id IS NULL`, operacja również jest cofana.

3. **Zapobieganie przypisaniu siebie jako przełożonego**  
   Trigger weryfikuje, czy pracownik nie jest swoim własnym przełożonym (`pracownik_id = przelozony_id`). W przypadku naruszenia tej zasady operacja zostaje anulowana.

Wszystkie naruszenia są zgłaszane za pomocą komunikatów błędów (`RAISERROR`), a odpowiednia transakcja zostaje wycofana, aby zachować spójność danych.


In [ ]:
CREATE TRIGGER trg_walidacja_pracownik_stanowisko
ON Osoby.Pracownik
AFTER INSERT, UPDATE
AS
BEGIN
    -- Sprawdzenie, czy więcej niż jeden pracownik ma stanowisko_id = 1
    IF EXISTS (
        SELECT 1
        FROM Osoby.Pracownik
        WHERE stanowisko_id = 1
        GROUP BY stanowisko_id
        HAVING COUNT(*) > 1
    )
    BEGIN
        RAISERROR ('Tylko jeden pracownik o stanowisku_id = 1 może mieć przelozony_id IS NULL.', 16, 1);
        ROLLBACK TRANSACTION;
        RETURN;
    END;

    -- Sprawdzenie, czy pracownicy z innymi stanowiskami mają przelozony_id IS NULL
    IF EXISTS (
        SELECT 1
        FROM inserted
        WHERE stanowisko_id != 1 AND przelozony_id IS NULL
    )
    BEGIN
        RAISERROR ('Pracownicy z innymi stanowiskami niż stanowisko_id = 1 muszą mieć przypisanego przełożonego.', 16, 1);
        ROLLBACK TRANSACTION;
        RETURN;
    END;

    -- Sprawdzenie, czy jakikolwiek pracownik jest swoim własnym przełożonym
    IF EXISTS (
        SELECT 1
        FROM inserted
        WHERE pracownik_id = przelozony_id
    )
    BEGIN
        RAISERROR ('Pracownik nie może być swoim własnym przełożonym.', 16, 1);
        ROLLBACK TRANSACTION;
        RETURN;
    END;
END;

GO

#### Trigger: walidacja maksymalnej liczby egzemplarzy w zamówieniu
Wyzwalacz kontrolujący, czy nie przekraczamy maksymalnego zamówienia, które wynosi 5 egzemplarzy.

In [ ]:
GO

CREATE TRIGGER trg_maksymalne_zamowienie
ON Zamowienia.ZamowienieEgzemplarz
AFTER INSERT, UPDATE
AS
BEGIN
    -- Deklaruję zmienną do przechowywania liczby egzemplarzy dla danego zamówienia
    DECLARE @ilosc INT;
    
    -- Sprawdzam liczbę egzemplarzy przypisanych do tego zamówienia
    SELECT @ilosc = COUNT(egzemplarz_id)
    FROM Zamowienia.ZamowienieEgzemplarz
    WHERE zamowienie_id IN (SELECT zamowienie_id FROM inserted); -- używam zamowienie_id z danych wstawianych/aktualizowanych

    -- Jeśli liczba egzemplarzy przekracza 5, cofam transakcję
    IF @ilosc > 5
    BEGIN
        RAISERROR ('Zamówienie nie może przekraczać 5 egzemplarzy.', 16, 1);
        ROLLBACK TRANSACTION;
    END;
END;

GO

### Trigger: Aktualizacja `pracownik_id` w tabeli `Zamowienie`

Wyzwalacz automatycznie tworzy powiązania między tabelą `Osoby.Pracownik` a tabelą `Zamowienia.Zamowienie` po zmianie danych w tabeli `Pracownik`. Składa się z 2 funkcjonalności:

1. **Wykrywanie zmian w `pracownik_id`**  
   Trigger sprawdza, czy zaktualizowano dane w tabeli `Pracownik`, które są powiązane z zamówieniami w tabeli `Zamowienie`.

2. **Aktualizacja `pracownik_id` w tabeli `Zamowienie`**  
   Jeżeli zmieniono `pracownik_id` w tabeli `Pracownik`, trigger automatycznie aktualizuje odpowiednie wiersze w tabeli `Zamowienie`, aby zachować spójność danych.

Dzięki temu mechanizmowi zmiany w tabeli `Pracownik` automatycznie propagują się do tabeli `Zamowienie`, eliminując potencjalne problemy z niespójnością danych. Byłem zmuszony wprowadzić ten wyzwalacz, ponieważ system nie dopuszczał ON UPDATE CASCASE dla encji `Zamowienie`.


### Komentarz
Możnaby wprowadzić wiele ciekawych elementów logiki poprzez wyzwalacze. Z uwagi na ramy czasowe, nie byłem niestety w stanie ich wprowadzić, miałem jednak kilka pomysłów: 
1. trigger automarycznie ustawiający datę zwortu
2. trigger dla zmiany statusu egzemplarz na wypożyczony
3. trigger dla automatycznego ustawienia data_zgloszenia w tabeli Naprawa
4. trigger dla automatycznego usuwania ZamowienieEgzemplarz po zwrocie

## Wprowadzenie danych

Kolejnym elementem było wprowadzenie danych, które wykonałem za pomocą `BULK INSERT` lub `INSERT INTO`, w zależności od liczby danych w tabelach. Dane generalnie mają sens, aczkolwiek nie zawsze. Należy pamiętać, że są to dane testowe, więc niekiedy pozbawione są logiki. Jak wspominałem przy okazji wyzwalaczy, dużym problemem było wygenerowanie dat, które spełniałyby wymagania logiczne stawiane przez triggery, przez co byłem zmuszony ostatecznie usunąć te ostatnie.

Dane wygenerowałem za pomocą Mockaroo, krótkiego skryptu napisanego w C++ (do tworzenia danych dla encji `ZamowienieEgzemplarz`) oraz za pomocą MS Excel.


### Wstawienie danych do encji `Osoba` za pomocą `BULK INSERT`
Ciekawym odkryciem było, że wyzwalacze nie działają w trakcie  `BULK INSERT`. Aby wymusić ich działanie konieczna jest komenda `FIRE_TRIGGERS`. Używam później często tej komendy przy imporcie.

In [ ]:
USE Wypozyczalnia;
GO

-- wstawiam dane do tabeli Osoba z pliku umieszczonego lokalnie
BULK INSERT Osoby.Osoba
FROM 'C:\data\osoby.csv'
WITH (
    FORMAT = 'CSV',         
    FIRSTROW = 2,           -- pomijam nagłówek
    FIELDTERMINATOR = ',',  -- serparator kolumn
    ROWTERMINATOR = '\n',   -- separator wierszy 
	FIRE_TRIGGERS           -- chcę aby wyzwalacze działały w trakcie importu
);

### Wstawienie danych do tabel z niewielką liczbą rekordów za pomocą `INSERT INTO`
W przypadku `INSERT INTO` triggery są domyślnie włączone i nie trzeba ich wymuszać.

In [ ]:
-- wstawiam dane do tabeli poprzez polecenie DML
INSERT INTO Osoby.Stanowisko(nazwa) 
VALUES
(N'Główny Kierownik'),
('Manager'),
('Kierownik'),
('Specjalista'),
('Pracownik Biurowy'),
('Technik'),
('Do aktualiazcji')

INSERT INTO Osoby.StatusZatrudnienia (nazwa_statusu) 
VALUES
('Aktywny'),
('Urlop'),
('Zwolniony'),
('Zawieszony'),
('Rekrutacja');

INSERT INTO Sprzet.Uprawnienie (nazwa) 
VALUES
('Do aktualizacji'),
('Patent Żeglarski'),
('Uprawnienia wspinaczkowe'),
('Uprawnienia rowerowe'),
('Pozwolenie na broń')

INSERT INTO Kategorie.PoraRoku (nazwa) VALUES
('Wiosna'),
('Lato'),
('Jesień'),
('Zima'),
(N'Przedwiośnie');

W przypadku Kategorii chciałem wymusić wprowadzone przeze mnie ID, ponieważ są kluczowe dla relacji. Musiałem w tym celu użyć komendy `SET IDENTITY_INSERT Kategorie.Kategoria ON`, z uwagi na to, że wprowadziłem auto-increment, czyli automatyczne nadawanie ID w tej tabeli (jak i w wielu innych, w istocie wszystkich poza tabelami asocjacyjnymi).

In [ ]:
SET IDENTITY_INSERT Kategorie.Kategoria ON;
INSERT INTO Kategorie.Kategoria (kategoria_id, nazwa, uprawnienie_id)
VALUES
(1,'Narciarstwo', NULL),
(2,'Żeglarstwo', NULL),
(3,'Tenis', NULL),
(4,'Kolarstwo', NULL),
(5,'Wspinaczka', NULL),
(6,'Letnie', NULL),
(7,'Sporty Olimpijskie', NULL),
(8,'Górski', NULL),
(9,'Wodny', 2);
SET IDENTITY_INSERT Kategorie.Kategoria OFF;

### Pozostałe Encje
Import danych do pozostałych encji przebiega klasycznie, podobnie jak w powyższych przypadkach, w związku z czym nie będę opisywał go szczegółowo. Różni się ewentualnie wprowadzeniem `CODEPAGE = '65001'` dla kodowanie UTF-8 oraz innymi separatorami i pierwszym wierszem.

In [ ]:
BULK INSERT Kategorie.KategoriaPoraRoku
FROM 'C:\data\poryroku_kat.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',    -- kodowanie UTF-8
    FIRSTROW = 2 ,
	FIRE_TRIGGERS
);


INSERT INTO Sprzet.Producent (nazwa, email, telefon, jezyk) 
VALUES
('Producent A', 'mail@producenta.pl', '123456789', 'Polski'),
('Producent B', 'rsg@producentb.com', '987654321', 'Francuski'),
('Producent C', 'elan@producentc.net', '543216789', 'Szwedzki'),
('Producent D', 'adidas@producentd.org', '678945321', 'Niemiecki'),
('Producent E', 'bienchi@prodE.eu', '789654123', N'Włoski');

INSERT INTO Serwis.StatusReklamacji (nazwa)
VALUES
('naprawiono'),
('odrzucono'),
('w trakcie naprawy'),
('rozpatrywana')

GO

-- wstawiam dane pracowników. Głowny kierownik jest kluczowy, dlatego wprowadzam go ręcznie. 
-- z uwagi na trigger, bulk insert mógłby nie zadziałać
SET IDENTITY_INSERT Osoby.Pracownik ON;

INSERT INTO Osoby.Pracownik 
    (pracownik_id, przelozony_id, stanowisko_id, pesel, adres, data_zatrudnienia, status_zatrudnienia, osoba_id)
VALUES 
    (1, NULL, 1, '91100337152', 'Kazikowskiego 20, 05-300 Mińsk Mazowiecki', '2015-01-18', 1, 2);

SET IDENTITY_INSERT Osoby.Pracownik OFF;

-- pozostali pracownicy wstawieni z opcją auto-increment wskazaną na poziomie kodu DDL
INSERT INTO Osoby.Pracownik 
    (przelozony_id, stanowisko_id, pesel, adres, data_zatrudnienia, osoba_id)
VALUES 
    (1, 2, '19681897386', 'Room 1860', '2022-07-23', 1),
    (1, 2, '19919460157', 'PO Box 94412', '2020-05-28', 3),
    (2, 3, '19740108973', 'PO Box 53194', '2021-02-02', 4),
    (2, 3, '19939943031', 'Apt 536', '2020-05-28', 5),
    (3, 3, '19560827075', 'Suite 1', '2024-05-27', 6),
    (4, 4, '19954704709', '18th Floor', '2023-08-28', 7),
    (5, 4, NULL, 'Suite 71', '2022-12-28', 8),
    (6, 4, '19897354554', 'Suite 76', '2022-12-28', 9),
    (6, 5, '19584710944', 'Apt 1931', '2021-02-27', 10),
    (6, 5, '19503315389', '2nd Floor', '2023-01-02', 11),
    (7, 6, NULL, 'PO Box 47030', '2024-09-24', 12),
    (7, 3, '19971501556', 'Apt 105', '2024-08-27', 13),
    (8, 2, '19606353605', 'Suite 3', '2021-09-12', 14),
    (9, 3, '19668714363', '19th Floor', '2023-06-12', 15);

-- Wstawianie danych do encji Klient
BULK INSERT Osoby.Klient
FROM 'C:\data\lista_klientow.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',    -- kodowanie UTF-8
    FIRSTROW = 2 ,
	FIRE_TRIGGERS
);

-- sprzet
BULK INSERT Sprzet.Sprzet
FROM 'C:\data\sprzet.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',     -- Kodowanie UTF-8
    FIRSTROW = 2,
	FIRE_TRIGGERS			-- chcę, aby triggery zadziałały już podczas bulk insert 
);

-- wgrywam dane do tabeli pośredniczącej SprzetKategoria
BULK INSERT Sprzet.SprzetKategoria
FROM 'C:\data\sprzet_kategoria.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',     -- Kodowanie UTF-8
    FIRSTROW = 2,
	FIRE_TRIGGERS			-- chcę, aby triggery zadziałały już podczas bulk insert 
);

-- wgrywam dane do tabeli egzemplarze
BULK INSERT Sprzet.Egzemplarz
FROM 'C:\data\egzemplarze.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',     -- Kodowanie UTF-8
    FIRSTROW = 1,
	FIRE_TRIGGERS			-- chcę, aby triggery zadziałały już podczas bulk insert 
);

-- wstawiam dane do encji Zamówienia. Dane z przedziału 2023-06-01 do 2024-06-01
BULK INSERT Zamowienia.Zamowienie
FROM 'C:\data\zamowienia.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',     -- Kodowanie UTF-8
    FIRSTROW = 2,
	FIRE_TRIGGERS			-- chcę, aby triggery zadziałały już podczas bulk insert 
);

-- wstawiam dane do encji zamówienie egzemplarz
BULK INSERT Zamowienia.ZamowienieEgzemplarz
FROM 'C:\data\zamowienia_szczegolowo.csv'
WITH (
    DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',     -- Kodowanie UTF-8
    FIRSTROW = 1
);

-- encja Reklamacje
BULK INSERT Serwis.Reklamacja
FROM 'C:\data\reklamacje.csv'
WITH (
	DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',   
    FIRSTROW = 2
);

-- encja Reklamacje
BULK INSERT Serwis.ReklamacjaEgzemplarz
FROM 'C:\data\reklamacje_szczegolowo.csv'
WITH (
	DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',   
    FIRSTROW = 2
);

-- encja Naprawa
BULK INSERT Serwis.Naprawa
FROM 'C:\data\naprawy.csv'
WITH (
	DATAFILETYPE = 'char',
    FIELDTERMINATOR = ';', 
    ROWTERMINATOR = '\n',  
    CODEPAGE = '65001',   
    FIRSTROW = 2
);


## Utworzenie procedur składowanych

W ramach projektu stworzyłem procedury składowane określone w wymaganiach. Projekt przewidywał również stworzenie zapytań generujących dane. Ponieważ procedury składowane zawierają te zapytania, zdecydowałem się nie tworzyć oddzielnych skryptów SQL dla zapytań i procedur. Zadanie zostało zrealizowane w następujący sposób:  

1. Stworzyłem zapytania (DQL) generujące odpowiednie dane.  
2. Następnie zawarłem te zapytania w procedurach składowanych, zamieniając kluczowe informacje (głównie w WHERE na parametry).

Gdybym realizował taki projekt w środowisku produkcyjnym, starałbym się opracować jak najwięcej procedur składowanych umożliwiających użytkownikom interakcję z bazą danych. Taki sposób działania pozwala wprowadzić dodatkową logikę, co zwiększa bezpieczeństwo oraz integralność danych.  

Na potrzeby tego projektu przygotowałem przykładowe procedury wymienione w opisie projektu.


### 1. wybór pracownika miesiąca czyli takiego, który obsłużył najwięcej zamówień w danym okresie czasu.

In [ ]:
CREATE PROCEDURE Osoby.PracownikMiesiaca
	@Od date,
	@Do date
AS
BEGIN
	SELECT
		TOP 1 p.pracownik_id AS pracownik_id,
		CONCAT(o.imie, ' ', o.nazwisko) AS N'imię i nazwisko',
		COUNT(z.zamowienie_id) AS N'liczba zamówień'
	FROM Osoby.Pracownik AS p
	JOIN Zamowienia.Zamowienie AS z ON z.pracownik_id = p.pracownik_id
	JOIN Osoby.Osoba AS o ON o.osoba_id = p.osoba_id
	WHERE z.data_zamowienia BETWEEN @Od AND @Do  -- zakres dat ustalony przez parametr
	GROUP BY p.pracownik_id, o.imie, o.nazwisko
	ORDER BY COUNT(z.zamowienie_id) DESC; 
END;

### 2. generowanie raportów o zamówieniach danego klienta w zadanym okresie czasu.

In [ ]:
CREATE PROCEDURE Zamowienia.ZamowieniaKlienta
	@KlientId int,
	@Od date,
	@Do date
AS
BEGIN
	SELECT
		z.zamowienie_id,
		z.cena_calkowita,
		z.pracownik_id AS opiekun,
		z.data_zamowienia
	FROM Zamowienia.Zamowienie AS z
	JOIN Osoby.Klient AS k ON k.klient_id = z.klient_id
	WHERE (z.data_zamowienia BETWEEN @Od AND @Do) AND k.klient_id = @KlientId;
END;


### 3. wyświetlenie rankingu sprzętu w danej kategorii

In [ ]:
CREATE PROCEDURE Sprzet.Ranking
	@KategoriaId int
AS
BEGIN
	SELECT
		k.nazwa as Kategoria,
		sk.ranking as Miejsce,
		s.nazwa as 'Nazwa Sprzętu'
	FROM Sprzet.SprzetKategoria sk
	JOIN Sprzet.Sprzet s ON s.sprzet_id = sk.sprzet_id
	JOIN Kategorie.Kategoria k ON k.kategoria_id = sk.kategoria_id
	WHERE sk.kategoria_id = 9
	ORDER BY sk.ranking ASC;
END;

### 4. znalezienie sprzętu, które zajmuje średnio najwyższe miejsce we wszystkich rankingach

In [ ]:
CREATE PROCEDURE Sprzet.AvgRanking
AS
BEGIN
	SELECT
		s.sprzet_id AS ID,
		s.nazwa AS nazwa_artykuły,
		FORMAT(AVG(CAST(sk.ranking AS decimal(10,4))), 'N2') AS sredni_ranking -- wprowadzam nieco bardziej czytelne formatowanie. Jednak rzutuje to decimal na varchar.
	FROM Sprzet.SprzetKategoria sk
	JOIN Sprzet.Sprzet s ON s.sprzet_id = sk.sprzet_id
	JOIN Kategorie.Kategoria k ON k.kategoria_id = sk.kategoria_id
	GROUP BY s.sprzet_id, s.nazwa
	ORDER BY AVG(CAST(sk.ranking AS decimal(10,4))) ASC; -- w celu poprawnego działania, eliminuję funkcję FORMAT() która powodowała rzutowanie
END;

### 5. Dodanie nowego sprzętu (z podaniem nazwy i typu)
Procedurę zaprojektowałem transakcyjnie w celu zapewnienia integralności danych (konieczne wstawienie danych do 2 encji). Ponadto umożliwiłem dodanie sprzętu z pewnymi wartościami domyślnymi.

In [ ]:
CREATE PROCEDURE Sprzet.DodajSprzet
    @Nazwa NVARCHAR(100),
    @Profesjonalny BIT,
    @Typ NVARCHAR(20),
    @CenaZaDobe DECIMAL(8,2),
    @ProducentId INT,
    @Rabat DECIMAL(5,2) = 0.00,		-- domyślnie 0.00
    @Opis NVARCHAR(255) = NULL,		-- domyślnie NULL
    @KategoriaId INT,
    @Ranking INT
AS
BEGIN
	BEGIN TRANSACTION -- chcę, aby wykonały się wszystkie instrukcje albo żadna
		BEGIN TRY
				-- Dodanie nowego sprzętu do tabeli Sprzet.Sprzet
				INSERT INTO Sprzet.Sprzet (nazwa, cena_za_dobe, profesjonalny, rabat, opis, producent_id)
				VALUES (@Nazwa, @CenaZaDobe, @Profesjonalny, @Rabat, @Opis, @ProducentId);

				-- Pobranie ID ostatnio dodanego sprzętu
				DECLARE @SprzetId INT;
				SET @SprzetId = SCOPE_IDENTITY();	-- pobieram ID sprzętu wstawionego powyżej

				-- dodaje sprzęt do tabeli SprzetKategoria w celu nadania mu kategorii
				IF (@KategoriaId IS NOT NULL)
				BEGIN
					INSERT INTO Sprzet.SprzetKategoria (sprzet_id, kategoria_id, ranking)
					VALUES (@SprzetId, @KategoriaId, @Ranking);
				END
			COMMIT TRANSACTION
		END TRY
    BEGIN CATCH
		ROLLBACK TRANSACTION -- cofam transakcję, jeżeli wystąpił chodziaż jeden błąd
        
		-- obsługa błędów
        DECLARE @ErrorMessage NVARCHAR(4000);
        DECLARE @ErrorSeverity INT;
        DECLARE @ErrorState INT;

        SELECT 
            @ErrorMessage = ERROR_MESSAGE(),
            @ErrorSeverity = ERROR_SEVERITY(),
            @ErrorState = ERROR_STATE();

        RAISERROR (@ErrorMessage, @ErrorSeverity, @ErrorState);
        RETURN;
    END CATCH
END;


### 6. wygenerowanie zestawienia wszystkich pracowników i ich szefów
Procedura składowana bez parametrów. Łączy się poprzez `JOIN` z samą sobą.

In [ ]:
CREATE PROCEDURE Osoby.ZestawieniePracownikow
AS
BEGIN
	SELECT 
		p.pracownik_id,
		CONCAT(o.imie, ' ', o.nazwisko) AS pracownik,
		p2.pracownik_id AS przelozony_id,
		CONCAT(o2.imie, ' ', o2.nazwisko) AS przelozony
	FROM Osoby.Pracownik p
	JOIN Osoby.Osoba o ON o.osoba_id = p.osoba_id
	-- self join z tabelą pracownik w celu uzyskania danych o przełożonym
	LEFT JOIN Osoby.Pracownik p2 ON p2.pracownik_id = p.przelozony_id
	-- ponowny join z tabelą osoba aby pobrać dane osobowe przełożonego
	LEFT JOIN Osoby.Osoba o2 ON p2.osoba_id = o2.osoba_id
	ORDER BY p.pracownik_id;
END;

### 7. Generowanie opisu sprzętu
Zadanie: otrzymanie opisu dla każdego rodzaju sprzętu, przy czym dla sprzętu górskiego opis ma uwzględniać nazwę, 
nazwę producenta, oraz porę roku w jakim sprzętu można używać, natomiast dla wodnego - nazwę, nazwę producenta, oraz informację o patencie

**Komentarz**: nie dodaję obsługi błedów, ponieważ nie spodziewam się, żeby takowe wystąpiły. Nie mniej w środowisku produkcyjnym, należałoby to dodać. Warto byłoby również dodać parametry, np. wybór kolumn do porównania. W tym przypadku tego nie robiłem, ponieważ zadanie wskazywało wyraźnie jedynie te 2 kategorie. Alternatywnie wykonałem wersję z jedną kolumną wskazującą opis (pory roku i patent w zależności od kategorii) jednak uznałem to rozwiązanie za nieczytelne i zmieniłem na 2 osobne kolumny. 

In [ ]:
CREATE PROCEDURE Sprzet.OpisSprzetu
AS
BEGIN
	SELECT
		s.nazwa AS Sprzęt,
		p.nazwa AS Producent,
		k.nazwa AS Kategoria,
		CASE
			WHEN k.kategoria_id = 8 THEN
				pr.nazwa -- nazwa pory roku jeżeli kategoria = 8 (czyli górski)
			ELSE
				NULL
		END AS 'Pora roku',
		CASE
			WHEN k.kategoria_id = 9 THEN
				u.nazwa -- nazwa patentu dla kategorii 9 (wodny)
			ELSE
				NULL
		END AS Patent
	FROM Sprzet.Sprzet s
	JOIN Sprzet.Producent p ON s.producent_id = p.producent_id
	JOIN Sprzet.SprzetKategoria sk ON s.sprzet_id = sk.sprzet_id
	JOIN Kategorie.Kategoria k ON sk.kategoria_id = k.kategoria_id
	LEFT JOIN Sprzet.Uprawnienie u ON u.uprawnienie_id = k.uprawnienie_id 
	LEFT JOIN Kategorie.KategoriaPoraRoku kpr ON kpr.kategoria_id = k.kategoria_id 
	LEFT JOIN Kategorie.PoraRoku pr ON pr.pora_roku_id = kpr.pora_roku_id
	WHERE k.kategoria_id IN (8, 9)
	ORDER BY k.nazwa, s.nazwa;
END;


## Wywołanie procedur składowanych
Wywołanie pierwszych 4 procedur składowanych:

In [ ]:
USE Wypozyczalnia;
GO

-- Zadanie 1: wywołuję pracedurę składowaną PracownikMIesiąca.
EXEC Osoby.PracownikMiesiaca @Od = '2023-07-01', @Do = '2023-07-30';

-- Zadanie 2: procedura składowana Zamówienia Klienta. Umożliwia generowanie raportów o zamówieniach danego klienta w zadanym okresie czasu.
EXEC Zamowienia.ZamowieniaKlienta @KlientId = 44, @Od = '2023-06-01', @Do = '2024-06-01';

-- Zadanie 3: wyświetlenie rankingu sprzętu w danej kategorii, w tym przypadku kategoria o id 8, czyli 'wodny'
EXEC Sprzet.Ranking 8;

-- Zadanie 4: znalezienie sprzętu, które zajmuje średnio najwyższe miejsce we wszystkich rankingach. W tym przypadku brak parametrów
EXEC Sprzet.AvgRanking;

#### Wywołanie procedury `Sprzet.DodajSprzet ` ze wszystkimi parametrami
Należy odpowiednio wprowadzić miejsce w rankingu, które nie może jeszcze być zajmowane w tej kategorii. Nie wprowadzałem logiki przesuwające w dół inne sprzęty z kategorii w przypadku wprowadzenia miejsca już zajmowanego. Możnaby to jednak uczynić w wersji produkcyjnej. Procedura działa transakcyjnie, jeżeli wstawienie rankingu się nie powiedzie, nie zostanie również wstawiony sprzęt w encji `Sprzet`.

In [ ]:
-- wersja a: wstawiam wszystkie parametry
EXEC Sprzet.DodajSprzet 
    @Nazwa = 'Kajak profesjonalny',
    @Profesjonalny = 1,
    @Typ = 'górski',
    @CenaZaDobe = 80.00,
    @ProducentId = 3,
    @Rabat = 5.00,
    @Opis = 'Rower profesjonalny górski',
    @KategoriaId = 8,
    @Ranking = 8;

#### Wywołanie procedury `Sprzet.DodajSprzet ` ze parametrami domyślnymi
Tym razem pomijam parametry `Opis` oraz `Rabat` - zostaną one uzupełnione wartościami domyślnymi ustawionymi podczas tworzenia procedury.

In [ ]:
EXEC Sprzet.DodajSprzet 
    @Nazwa = 'Kajak profesjonalny',
    @Profesjonalny = 1,
    @Typ = 'wodny',
    @CenaZaDobe = 80.00,
    @ProducentId = 3,
    @KategoriaId = 9,
    @Ranking = 6;

#### Ostatnie 2 procedury składowane

In [ ]:
-- Zadanie 6: wygenerowanie zestawienia wszystkich pracowników i ich szefów. Brak parametrów.
EXEC Osoby.ZestawieniePracownikow;

/* Zadanie 7: otrzymanie opisu dla każdego rodzaju sprzętu, przy czym dla sprzętu górskiego opis ma uwzględniać nazwę, 
nazwę producenta, oraz porę roku w jakim sprzętu można używać, natomiast dla wodnego - nazwę, nazwę producenta, oraz informację o patencie.
Aby zachować czytelność, rozdzieliłem te informacje na 2 dodatkowe kolumny, zależne od typu (nazywanego u mnie kategorią).
*/
EXEC Sprzet.OpisSprzetu;

## Podsumowanie

W pliku opracowałem model relacyjnej bazy danych dla wypożyczalni sprzętu sportowego, uwzględniając opis projektu, modelowanie danych oraz implementację rozwiązania.  

Modelowanie obejmowało zarówno model konceptualny, jak i implementacyjny, gdzie szczegółowo przedstawiłem podział na schematy oraz budowę modelu. Opisałem również największe wyzwania, z jakimi musiałem się zmierzyć podczas jego tworzenia.  

W sekcji implementacji modelu pokazałem proces tworzenia bazy danych. Zawierał on kroki takie jak: usunięcie istniejącej bazy, utworzenie nowej bazy z określoną lokalizacją, stworzenie użytkownika, schematów oraz nadanie odpowiednich uprawnień. Opisałem także tworzenie encji i indeksów.  

Dodatkowo zaimplementowałem wyzwalacze (triggery), które m.in. walidują dane w tabelach i aktualizują odpowiednie wartości w bazie. Opisałem również ich cel oraz sposób działania.  

Przedstawiłem proces wprowadzania danych do bazy, używając zarówno polecenia `BULK INSERT` dla większych zbiorów danych, jak i klasycznego `INSERT INTO` dla mniejszych.  

Stworzyłem procedury składowane, które realizują kluczowe funkcje opisane w wymaganiach projektu. Wśród nich znalazły się procedury do generowania raportów, wyboru pracownika miesiąca, wyświetlania rankingów sprzętu, czy dodawania nowego sprzętu do bazy. Każda procedura została opisana wraz z przykładem jej wywołania.  

Na koniec omówiłem sposób wywoływania procedur oraz podsumowałem efekty wykonanych prac, wskazując na ich praktyczne zastosowanie i potencjalne korzyści w środowisku produkcyjnym.

Projekt pomógł mi przećwiczyć szeroki zakres pojęć i zapytań języka TSQL w praktyce. Największym wyzwaniem było modelowanie bazy danych oraz tworzenie procedur składowanych (`DodajSprzet` oraz `OpisSprzetu`). Sporo problemów sprawiło mi również wprowadzenie logiki związanej z faktem, że w bazie może istnieć jedynie jeden pracownik który nie ma przełożonego. Musiałem podjąc decyzję, czy w atrybucie `przelozony` mogą pojawić się wartości `NULL` czy nie, oraz jak to obsłużyć. Finalnie zdecydowałem się na wyzwalacz, którego stworzenie zajeło mi sporo czasu: `trg_walidacja_pracownik_stanowisko`.

Dziękuję za uwage i pozdrawiam.
